# ⚙️ What is aiosql library?

Aiosql is a Python library that simplifies the writing of SQL queries in separate files from your main Python project code. These queries, stored in SQL files, are then transformed into methods within a Python object. 
Another notable feature of Aiosql is its ability to generate dynamic methods that accept parameters, enabling flexible query execution and effective interaction with the underlying database. This separation of SQL queries from the main Python code promotes cleaner and more modular code, enhancing project readability and maintainability


<center>

![aiosql-feature](img/aiosql-0.png)

</center>

# ⚙️ Key Features of Aiosql Library

Below, I will share a series of features that this library already has or can have based on its usage:
* Provides **CRUD functionality** (Create: Insert, Read: Select, Update , Delete) for database operations.
* **Separates Python code** from SQL code, making it easier to locate queries within projects with multiple databases.
* Each query can be assigned a **descriptive name and docstring**, similar to Python functions, enabling documentation of the query.
* Facilitates the creation of **a query catalog** within the project, aiding in identification based on entities, databases, or other grouping criteria.
* Enables easy generation of **dynamic queries** with the ability to pass dynamic values and modify them as needed.

<center>

![aiosql-feature](img/aiosql-1.png)

</center>

---

# ⚙️ Aiosql Tutorial

## 🔧 Prerequisites
* 🐳 Docker
* 🐙 Docker Compose
* 🐍 Install python libraries: !pip install aiosql pandas 

## 🚀 Quick Start

### 🛠️Create a postgres database
* 1️⃣ - **Create postgres database** → Execute in the terminal: `docker-compose -f docker-compose.yml up --build`
* 2️⃣ - **Check if your container is running** → Execute in the terminal: `docker ps`
* 3️⃣ - **Load the csv files** → Execute the following command for load the csv file in the container: `python3 etl.py`
  
### 🏥 Hospital Data
To implement aiosql, we will use the dataset from Synthea, which simulates a hospital database. These synthetic data are generated from a simulation considering various variables of a population in Massachusetts. 
From this dataset, we will use the tables: `conditions`, `encounters`, and `patients`. 

### 👥 User stories
To make this example more real we are going to make 3 use cases:
* 1️⃣ - As a **data analyst**, I want to be able to retrieve a list of patients whose visit count is above the 90th percentile, so that I can identify the most active patients in the clinic. Additionally, I want this percentile to be configurable for easy adjustment in the future.
* 2️⃣ - As a **researcher or data analyst**, I want to access the data of patients who have been diagnosed with the 10 most frequent diagnoses in a period of time, in order to analyze trends and improve the quality of medical care.
* 3️⃣ - As a **marketing analyst**, I want to create a table for patient satisfaction surveys, so that I can gather feedback on the quality of care and take measures to improve it.

### Implementation
Based on the user stories that we are going to create, we will define two files in which we will load the queries and scripts that we need to execute:
* **patients.sql**: where we have all the queries related to recovering patient data.
* **visits.sql**: where we have all the queries related to visits, such as surveys.

Therefore in our project we are going to have this structure of folders and files
```bash
- 📁 db
	- 📁 queries
		- 📄 patients.sql
		- 📄 visits.sql
- 📄 main.ipynb
```

---

In this way we are isolating the python code from the sql code, in our case we are going to implement this **🐍python code** in a notebook in such a way as to make its explanation easier.

---



1. Import python libraries 

In [104]:
import aiosql
import psycopg2
import pandas as pd

2. Import the SQL queries and configure the database driver

    > In this project, the SQL queries are located in the **'db/queries'** directory and **'psycopg2'** is the PostgreSQL database adapter.

In [232]:
sql = aiosql.from_path('src/db/queries', 'psycopg2')

3. Create the connection to the PostgreSQL database.

In [ ]:
postgres_secrets = {'host': 'localhost','port': 5432, 'user': 'postgres', 'password': 'postgres', 'dbname': 'postgres'}
conn = psycopg2.connect(**postgres_secrets)
conn.autocommit = True

### 👥 User story I: Static Values
> As a **data analyst**, I want to be able to retrieve a `list of patients` whose visit count is `above the 90th percentile`, so that I can identify the most active patients in the clinic. Additionally, I want this __percentile to be configurable__ for easy adjustment in the future.

---

Based on this user story, we will first create one that allows generating a query to retrieve the list of patients with a visit frequency above the 90th percentile.

1. In the sql file we have the query for the first user story

    `db/queries/patients.sql`

    ```sql
    -- name: fn_get_patients_adove_90th_percentile
    -- get all the patients that have more visits than the 90th percentile of visits. All this data is stored in encounters table.
    WITH patient_visits AS (
        SELECT
            patient,
            COUNT(*) AS visit_count
        FROM
            hospital.encounters
        GROUP BY
            patient
    ),
    percentil_n AS (
        SELECT
            percentile_cont(0.9) WITHIN GROUP (ORDER BY visit_count) AS p_visits
        FROM
            patient_visits
    )
    SELECT 
        pv.patient, 
        pv.visit_count
    FROM 
        patient_visits pv
    CROSS JOIN 
        percentil_n  pn
    WHERE 
        pv.visit_count >= pn.p_visits;
    ```

2. Execute the 'fn_get_patients_above_90th_percentile' SQL function using the database connection 'conn'.
    > The function returns a list of tuples representing patients whose visit count is above the 90th percentile.

In [ ]:
response = sql.fn_get_patients_above_90th_percentile(conn)

3. Now we can convert the response object into a pandas DataFrame for easier data manipulation
    > The column names ('patient_id' and 'num_visit') are added manually because aiosql only returns the query result as a list of tuples without column names.


In [181]:
data = pd.DataFrame([item for item in response], columns=['patient_id', 'num_visit'])
# Display the DataFrame.
data

,patient_id,num_visit
0,8b9a93f6-3df3-203d-932f-f456e00d2c01,414
1,e0b86a58-ba53-3d16-0824-390f4507aaa6,115
2,ce00a1df-19ec-39db-e163-7022fdaf9103,87
3,e388a0a2-d98c-55f7-e010-9901916b1c41,123
4,6268d952-3a59-9bee-06ca-26740680f80a,401


  > if we want to see the query, we can use the following code

In [187]:
print(sql.fn_get_patients_adove_90th_percentile.sql)

WITH patient_visits AS (
SELECT
patient,
COUNT(*) AS visit_count
FROM
hospital.encounters
GROUP BY
patient
),
percentil_n AS (
SELECT
percentile_cont(0.9) WITHIN GROUP (ORDER BY visit_count) AS p_visits
FROM
patient_visits
)
SELECT
pv.patient,
pv.visit_count
FROM
patient_visits pv
CROSS JOIN
percentil_n  pn
WHERE
pv.visit_count >= pn.p_visits;


### 👥 User story I: Dynamic Values
> As a **data analyst**, I want to be able to retrieve a `list of patients` whose visit count is above the 90th percentile, so that I can identify the most active patients in the clinic. Additionally, `I want this percentile to be configurable` for easy adjustment in the future.


---

Now, we are going to create another query that allows us to accept different percentile values so that the query can be dynamically modified based on the values passed. In our case, we are going to provide an example of obtaining the list of patients that exceed the 75th percentile.

> Notice that we now have a dynamic variable called **percentile_value**

 ```sql
    
    -- name: fn_get_patients_above_n_percentile
    -- This query retrieves all patients whose visit count exceeds the nth percentile of all visit counts 
    WITH patient_visits AS (
        SELECT
            patient,
            COUNT(*) AS visit_count
        FROM
            hospital.encounters
        GROUP BY
            patient
    ),
    percentil_n AS (
        SELECT
            percentile_cont(:percentil_value) WITHIN GROUP (ORDER BY visit_count) AS p_visits
        FROM
            patient_visits
    )
    SELECT 
        pv.patient, 
        pv.visit_count
    FROM 
        patient_visits pv
    CROSS JOIN 
        percentil_n  pn
    WHERE 
        pv.visit_count >= pn.p_visits;
```


1. This following code executes a dynamic SQL query that accepts different percentile values as input.

In [197]:
# In this case, we're getting patients above the 75th percentile.
response = sql.fn_get_patients_above_n_percentile(conn, percentil_value=0.75)
data = pd.DataFrame([item for item in response], columns=['patient_id', 'num_visit'])

data


,patient_id,num_visit
0,8b9a93f6-3df3-203d-932f-f456e00d2c01,414
1,dbb11a89-9cb5-044e-f27d-3985b6bddcca,71
2,46cd115f-10aa-7613-18ff-ca767df60987,73
3,3ff2f362-af28-9555-1f8e-71d1a60b7289,86
4,91356f3d-7765-7eac-25ef-6ead6b408811,55
5,6f421a1a-ad37-b024-0f04-72eb9f786ee4,64
6,caa8478c-17ea-7c64-553c-16ad56f4a45f,54
7,e0b86a58-ba53-3d16-0824-390f4507aaa6,115
8,ce00a1df-19ec-39db-e163-7022fdaf9103,87
9,e388a0a2-d98c-55f7-e010-9901916b1c41,123


### Other operations

1. To determine the type of SQL operation performed by the 'fn_get_patients_above_n_percentile' function
    > we can access the 'operation' attribute of the function and then its 'name' attribute, and this will return the name of the SQL operation (e.g., 'select', 'insert', 'update', 'delete').

In [203]:
sql.fn_get_patients_above_n_percentile.operation.name

'SELECT'

2. To access the documentation (docstring) of the 'fn_get_patients_above_n_percentile' SQL function

In [205]:
sql.fn_get_patients_above_n_percentile.__doc__

'This query retrieves all patients whose visit count exceeds the nth percentile of all visit counts'

3. By using the **'?'** command, we can access detailed information about the `'fn_get_patients_above_n_percentile'` function. This includes its parameters (in this case, `'percentile_value'`), its docstring, and the file path where it's defined.

In [206]:
?sql.fn_get_patients_above_n_percentile

Signature: sql.fn_get_patients_above_n_percentile(*, percentil_value)
Docstring: This query retrieves all patients whose visit count exceeds the nth percentile of all visit counts
File:      ~/Desktop/romi/Personal/blog/github/publications/aiosql/postgres/src/db/queries/patients.sql
Type:      method


### 👥 User stories II

> As a **researcher or data analyst**, I want to access the data of patients who have been diagnosed with the 10 most frequent diagnoses in a period of time, in order to analyze trends and improve the quality of medical care.

---

To resolve this user story, we will create a query that retrieves patients with the most common conditions within a specified time period. This query will be dynamic, allowing for future variations in the number of conditions of interest. 
It will accept three parameters: 
- **'num_condition'** will allow us to limit the number of conditions we're interested in (e.g., the top 10 most common conditions).
- **'period_start_date'** and **'period_start_end'** will define the time window for which we want to retrieve data.

```sql
-- name: fn_get_patients_top_conditions
-- Get patients with top conditions for a given period of time, the patients are sorted by the number of days they had the condition and the source of the data is the hospital schema.
with top_n_conditions as(
SELECT  code, description, COUNT(*) 
     FROM hospital.CONDITIONS 
     GROUP BY  code,description 
     ORDER BY COUNT(*) DESC 
     LIMIT  :num_condition
),
top_n_condition_patients as (
SELECT 
    p.ID, 
    p.FIRST, 
    p.LAST, 
    p.CITY, 
    p.GENDER, 
    EXTRACT(YEAR FROM AGE(p.BIRTHDATE)) AS age,
    c.start condition_start_date,
    c.stop condition_stop_date,
    EXTRACT(DAY FROM (c.stop - c.start )) AS condition_days, 
    c.encounter,
    c.code,
    c.description
    from hospital.patients p 
    inner join hospital.conditions c  on c.patient = p.id
    inner join top_n_conditions t on t.code=c.code
)
select * 
    from top_n_condition_patients
    where condition_start_date between :period_start_date and :period_start_end;
```


In [212]:
response = sql.fn_get_patients_top_conditions(conn, num_condition_days=10, 
                                        period_start_date='2022-01-01', 
                                        period_start_end='2022-12-31')

column_name=['id', 'first','last','city','gender',
'age','condition_start_date','condition_stop_date','condition_days','encounter','code','description']

data = pd.DataFrame([item for item in response], columns=column_name)
data.head()


,id,first,last,city,gender,age,condition_start_date,condition_stop_date,condition_days,encounter,code,description
0,3206f7cc-bc9a-b870-da96-718e67ef99c0,Morgan564,Feil794,Waltham,M,59,2022-01-07,2022-01-07,0,de679f99-b5bf-14ea-c3c4-bff41ec8846a,314529007,Medication review due (situation)
1,fcef92a1-6fbd-ec7f-0fa7-ac3650e79e00,Dannie881,Wolf938,Canton,M,12,2022-06-12,2023-11-05,511,d27619e2-5f1f-928b-7a58-c7c775841078,314529007,Medication review due (situation)
2,b3211cb0-c34b-eb9c-23b6-58726ad437ec,Alphonso102,Dicki44,Walpole,M,61,2022-06-16,2023-06-15,364,d6602a0a-67a0-3a0a-2c4e-bce9598ca221,314529007,Medication review due (situation)
3,b3211cb0-c34b-eb9c-23b6-58726ad437ec,Alphonso102,Dicki44,Walpole,M,61,2022-06-16,2023-06-15,364,d6602a0a-67a0-3a0a-2c4e-bce9598ca221,160903007,Full-time employment (finding)
4,baed88fc-8726-ee3a-47bd-b67d38c63df5,Vince741,Stamm704,Dalton,M,68,2022-03-28,2022-09-26,182,393eb61e-8a9a-e26c-26e2-34531b1affc5,160903007,Full-time employment (finding)


### 👥 User story III
As a **marketing analyst**, I want to create a table for patient satisfaction surveys, so that I can gather feedback on the quality of care and take measures to improve it.

---

Now we are going to create the table using aiosql, if you look at our code in SQL you will see that a # symbol is added, these symbols are used by aiosql to identify the different operations that must be performed.

```sql
-- name: fn_create_survey_table#
CREATE TABLE HOSPITAL.VISIT_SURVEY(
    ID SERIAL PRIMARY KEY,
    PATIENT_ID VARCHAR(50),
    SURVEY_DATE TIMESTAMP,
    RATING INT,
    COMMENTS TEXT,
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

<center>

![aiosql](img/aiosql-2.png)

</center>


1. Execute the 'fn_create_survey_table' SQL function to create a new table in the database.

In [224]:
sql.fn_create_survey_table(conn)


'CREATE TABLE'

2. Once the table is created we are going to use the following insert statement to be able to insert a review of a patient

```sql
-- name: fn_add_one_visit_survey<!
insert into HOSPITAL.VISIT_SURVEY(PATIENT_ID,SURVEY_DATE,RATING,COMMENTS) 
    values (:patient_id, :survey_date, :rating,:comments) returning ID;
```

---

> **⚠️ Disclaimer**: During the coding of this tutorial, I used the insert statement without a return and encountered an error due to its absence. (The version of aiosql I am using is 10.1)
This 'returning ID' allows us to retrieve the value assigned to the 'id' column within the 'hospital_visit_survey' table when the insert operation is performed.

---

In [231]:
# Add a new visit survey record 

sql.fn_add_one_visit_survey(conn, 
                            patient_id='8b9a93f6-3df3-203d-932f-f456e00d2c01', 
                            survey_date='2022-01-01', 
                            rating=5,
                            comments='This is a great hospital!' )


1

Next we will be able to make a query and validate if the review was loaded into the table

In [235]:
response = sql.fn_get_visit_survey(conn)
column_name=['id','patient_id', 'survey_date','rating','comments','created_at']
data = pd.DataFrame([item for item in response], columns=column_name)
data.head()


,id,patient_id,survey_date,rating,comments,created_at
0,1,8b9a93f6-3df3-203d-932f-f456e00d2c01,2022-01-01,5,This is a great hospital!,2024-05-19 20:54:52.813714


3. Now we will utilize a new insert statement to load multiple reviews, which are stored in a list of dictionaries (each dictionary in Python corresponds to a review). To accomplish this, we will employ a similar query but we need to modify its name

```sql
    -- name: fn_add_many_visit_survey*!
    insert into HOSPITAL.VISIT_SURVEY(PATIENT_ID,SURVEY_DATE,RATING,COMMENTS) 
        values (:patient_id, :survey_date, :rating ,:comments) returning ID;
```

In [237]:
# Add several visit survey records
response_survey = [
    {
        'patient_id': '8b9a93f6-3df3-203d-932f-f456e00d2c01',
        'survey_date': '2022-01-01',
        'rating': 3,
        'comments': 'The service was good. But the waiting time was a bit long.'
    },
    {
        'patient_id': '7c8a93f6-4df3-203d-932f-f456e00d2c02',
        'survey_date': '2022-02-01',
        'rating': 4,
        'comments': 'The staff was very helpful!'
    },
    {
        'patient_id': '6b7a93f6-5ef3-203d-932f-f456e00d2c03',
        'survey_date': '2022-03-01',
        'rating': 3,
        'comments': 'The waiting time was a bit long.'
    }
]


sql.fn_add_many_visit_survey(conn, response_survey)

3

In [238]:
response = sql.fn_get_visit_survey(conn)
column_name=['id','patient_id', 'survey_date','rating','comments','created_at']
data = pd.DataFrame([item for item in response], columns=column_name)
data.head()

,id,patient_id,survey_date,rating,comments,created_at
0,1,8b9a93f6-3df3-203d-932f-f456e00d2c01,2022-01-01,5,This is a great hospital!,2024-05-19 20:54:52.813714
1,2,8b9a93f6-3df3-203d-932f-f456e00d2c01,2022-01-01,3,The service was good. But the waiting time was...,2024-05-19 21:12:17.439324
2,3,7c8a93f6-4df3-203d-932f-f456e00d2c02,2022-02-01,4,The staff was very helpful!,2024-05-19 21:12:17.649318
3,4,6b7a93f6-5ef3-203d-932f-f456e00d2c03,2022-03-01,3,The waiting time was a bit long.,2024-05-19 21:12:17.678712


# Queries catalog of your project

At the beginning of the tutorial, I mentioned the possibility of creating a query catalog for your project. While this functionality is not directly provided by this library, with the following Python code, you can create a dataframe containing the name of your queries, their operation, the code of your query, the docstring with its description, and the parameters that make your query dynamic.

In [306]:
def get_catalog_methods(sql) -> pd.DataFrame:
    fn = [item for item in sql.available_queries if not item.endswith('_cursor')]
    output = []

    for item in fn:
        query = getattr(sql, item)
        operation = query.operation.name
        query_sql = query.sql
        description = query.__doc__
        parameters = list(query.__signature__.parameters.keys())
        output.append((item, operation, query_sql, description, parameters))

    data = pd.DataFrame(output, columns=['function_name', 'operation', 'query', 'description', 'parameters'])
    return data

In [307]:
sql = aiosql.from_path('src/db/queries', 'psycopg2')
get_catalog_methods(sql=sql)

,function_name,operation,query,description,parameters
0,fn_add_many_visit_survey,INSERT_UPDATE_DELETE_MANY,"insert into HOSPITAL.VISIT_SURVEY(PATIENT_ID,S...",,"[self, patient_id, survey_date, rating, comments]"
1,fn_add_one_visit_survey,INSERT_RETURNING,"insert into HOSPITAL.VISIT_SURVEY(PATIENT_ID,S...",,"[self, patient_id, survey_date, rating, comments]"
2,fn_create_survey_table,SCRIPT,CREATE TABLE HOSPITAL.VISIT_SURVEY(\nID SERIAL...,,[self]
3,fn_get_patients_above_n_percentile,SELECT,"WITH patient_visits AS (\nSELECT\npatient,\nCO...",This query retrieves all patients whose visit ...,"[self, percentil_value]"
4,fn_get_patients_adove_90th_percentile,SELECT,"WITH patient_visits AS (\nSELECT\npatient,\nCO...",get all the patients that have more visits tha...,[self]
5,fn_get_patients_top_conditions,SELECT,"with top_n_conditions as(\nSELECT code, descr...",Get patients with top conditions for a given p...,"[self, num_condition_days, period_start_date, ..."
6,fn_get_visit_survey,SELECT,SELECT * FROM HOSPITAL.VISIT_SURVEY;,,[self]
